## Midterm

#### Objective: Conduct analysis on the provided data to identify the characteristics of users who check out, and those who don't.

##### Part one: Data Cleaning & Wrangling

How did you prepare your data for analysis? Describe your data cleaning and preparation approaches, and why these were the best choices.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
dt = pd.read_csv('/Users/jiawenli/Desktop/midterm_data.csv')

In [3]:
dt.head()

,userID,age,sessions,time_spent,pages_visited,cart_items,cart_value,checkout_status,device,location
0,1,62,3,13.295129,2,20,199.21,0,Desktop,Location 1
1,2,65,2,21.686405,3,21,294.82,0,Desktop,Location 3
2,3,18,7,17.135220,4,23,240.87,0,Desktop,Location 4
3,4,21,9,29.589311,8,16,183.78,1,Desktop,Location 2
4,5,21,3,25.948584,4,10,59.51,1,Tablet,Location 4


In [9]:
dt['userID'].unique()

array([   1,    2,    3, ..., 4998, 4999, 5000])

In [10]:
# Get a general idea of the dataset we have
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   userID           5000 non-null   int64  
 1   age              5000 non-null   int64  
 2   sessions         5000 non-null   int64  
 3   time_spent       5000 non-null   float64
 4   pages_visited    5000 non-null   int64  
 5   cart_items       5000 non-null   int64  
 6   cart_value       5000 non-null   float64
 7   checkout_status  5000 non-null   int64  
 8   device           4900 non-null   object 
 9   location         4970 non-null   object 
dtypes: float64(2), int64(6), object(2)
memory usage: 390.8+ KB


1. Drop duplicates: By dropping duplicates, we would avoid doing repetitive work and it also saves meemory.

In [11]:
dt.drop_duplicates()

,userID,age,sessions,time_spent,pages_visited,cart_items,cart_value,checkout_status,device,location
0,1,62,3,13.295129,2,20,199.21,0,Desktop,Location 1
1,2,65,2,21.686405,3,21,294.82,0,Desktop,Location 3
2,3,18,7,17.135220,4,23,240.87,0,Desktop,Location 4
3,4,21,9,29.589311,8,16,183.78,1,Desktop,Location 2
4,5,21,3,25.948584,4,10,59.51,1,Tablet,Location 4
...,...,...,...,...,...,...,...,...,...,...
4995,4996,48,8,45.000000,11,25,278.92,0,Tablet,Location 5
4996,4997,31,4,44.700747,9,12,78.97,0,Tablet,Location 2
4997,4998,34,5,27.146624,7,19,139.42,1,Desktop,Location 4
4998,4999,58,1,45.000000,11,24,203.42,0,Desktop,Location 5


We originally have 5000 entries and after dropping duplicates, we still have 5000 entries, which indicates that there is no duplicates in the dataset. This is great.

2. Dealing with outliers
In our dataset, if there is any outliers, we would like to include them in our dataset since they might be providing any potential insights toward our analysis.

3. Check for missing values

In [13]:
dt.isnull().sum()

userID               0
age                  0
sessions             0
time_spent           0
pages_visited        0
cart_items           0
cart_value           0
checkout_status      0
device             100
location            30
dtype: int64

As we can see here, we only have missing values in the device, and location columns.
We have 100 missing values in the device column, and 30 missing values in the location column.

The method I will be using is to impute the missing value for both device and location. I will use the mode to impute the missing value for both columns. The reason for using mode is because mode could represents the preferences of customers in some extent.

In [19]:
# Check for unique value within the location column
# Find the location that appears the most
dt['location'].unique()

array(['Location 1', 'Location 3', 'Location 4', 'Location 2',
       'Location 5', nan], dtype=object)

In [20]:
dt['location'].value_counts()

Location 4    1056
Location 3    1014
Location 1     979
Location 5     971
Location 2     950
Name: location, dtype: int64

It appears that 'Location 4' appears the most in the location column with a number of 1056. Then I will fill all of the missing values in location column with the value of 'Location 4'.

In [29]:
dt['location'].fillna('Location 4', inplace=True)

In [30]:
# Now, let's deal with device column. Similarly, we find out all the unique value in device column
# Then, figure out which device appears the most
dt['device'].unique()

array(['Desktop', 'Tablet', 'Mobile'], dtype=object)

In [31]:
dt['device'].value_counts()

Desktop    2638
Mobile     1428
Tablet      934
Name: device, dtype: int64

It appears that most of users were using Desktops. Then we could impute the missing value using 'Desktop'.

In [32]:
dt['device'].fillna('Desktop', inplace=True)

In [33]:
# After dealing with missing value, double check and make sure there is no missing value
dt.isnull().sum()

userID             0
age                0
sessions           0
time_spent         0
pages_visited      0
cart_items         0
cart_value         0
checkout_status    0
device             0
location           0
dtype: int64

4. Last step for data cleaning is to make sure all of the columns are having appropriate data types.

In [34]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   userID           5000 non-null   int64  
 1   age              5000 non-null   int64  
 2   sessions         5000 non-null   int64  
 3   time_spent       5000 non-null   float64
 4   pages_visited    5000 non-null   int64  
 5   cart_items       5000 non-null   int64  
 6   cart_value       5000 non-null   float64
 7   checkout_status  5000 non-null   int64  
 8   device           5000 non-null   object 
 9   location         5000 non-null   object 
dtypes: float64(2), int64(6), object(2)
memory usage: 390.8+ KB


We could perform feature engineering by converting the data type of both device and location from object to category, this could helps us save memory and improve computational efficiency.

In [36]:
dt['location'] = dt['location'].astype('category')
dt['device'] = dt['device'].astype('category')
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   userID           5000 non-null   int64   
 1   age              5000 non-null   int64   
 2   sessions         5000 non-null   int64   
 3   time_spent       5000 non-null   float64 
 4   pages_visited    5000 non-null   int64   
 5   cart_items       5000 non-null   int64   
 6   cart_value       5000 non-null   float64 
 7   checkout_status  5000 non-null   int64   
 8   device           5000 non-null   category
 9   location         5000 non-null   category
dtypes: category(2), float64(2), int64(6)
memory usage: 322.7 KB


##### Part two:
##### The problem: Understanding the reason why less than 30% users actually check out after they have added items to their cart

1. Firstly, we need to make sure that users from differrent locations, with different devices and different age does not draw significant impact on the outcome of checking out or not. For us to perform a meaningful experimental design, we need to ensure users have a similar starting characteristics.

Perform ANOVA to see if there is a significant diffenrence between all 5 locations and checkout_status. If there is no significant difference, then we should have a high p-value.

In [40]:
# Perform ANOVA test to see if all 5 locations would draw significant impact on the checkout_status
loc_1 = dt[dt['location'] == 'Location 1']['checkout_status']
loc_2 = dt[dt['location'] == 'Location 2']['checkout_status']
loc_3 = dt[dt['location'] == 'Location 3']['checkout_status']
loc_4 = dt[dt['location'] == 'Location 4']['checkout_status']
loc_5 = dt[dt['location'] == 'Location 5']['checkout_status']

F_stats, p_value = stats.f_oneway(loc_1, loc_2, loc_3, loc_4, loc_5)

print('ANOVA Test')
print('F-statistics: {:.5f}'.format(F_stats))
print('P-value: {:.5f}'.format(p_value))

ANOVA Test
F-statistics: 0.54913
P-value: 0.69968


Based on the p-value of 0.69968, which is high, we fail to reject to our null hypothesis. Thus, there is no significant difference between the mean of checkout_status from different locations.

Next, we perform ANOVA test to see if there is a significant diffenrence on checkout_status from users with different devices. If there is no significant difference, then we should also have a high p-value.

In [50]:
# Perform ANOVA test to see if different devices would draw significant impact on the checkout_status
desktop = dt[dt['device'] == 'Desktop']['checkout_status']
tablet = dt[dt['device'] == 'Tablet']['checkout_status']
mobile = dt[dt['device'] == 'Mobile']['checkout_status']

F_stats, p_value = stats.f_oneway(desktop, tablet, mobile)

print('ANOVA Test')
print('F-statistics: {:.5f}'.format(F_stats))
print('P-value: {:.5f}'.format(p_value))

ANOVA Test
F-statistics: 1.66016
P-value: 0.19021


The p-value of 0.19021 suggests that there is no significant difference between the mean of checkout_status from users with different devices.

Next, we could perform feature engineering again to group users into different age segments by adding a new column could age_segment.

In [43]:
dt['age'].describe()

count    5000.000000
mean       41.538200
std        14.042734
min        18.000000
25%        29.000000
50%        42.000000
75%        54.000000
max        65.000000
Name: age, dtype: float64

Based on the age insight we generate above, we could filter users by '18-25', '26-40', '41-55', and '56+'

In [44]:
def application_func_1(x):
    if x >= 18 and x <= 25:
        return '18-25'
    elif x >= 26 and x <= 40:
        return '26-40'
    elif x >= 41 and x <= 55:
        return '41-55'
    else:
        return '56+'

In [47]:
dt['age_segment'] = dt['age'].apply(application_func_1)
dt['age_segment'] = dt['age_segment'].astype('category')

In [48]:
dt.head()

,userID,age,sessions,time_spent,pages_visited,cart_items,cart_value,checkout_status,device,location,age_segment
0,1,62,3,13.295129,2,20,199.21,0,Desktop,Location 1,56+
1,2,65,2,21.686405,3,21,294.82,0,Desktop,Location 3,56+
2,3,18,7,17.135220,4,23,240.87,0,Desktop,Location 4,18-25
3,4,21,9,29.589311,8,16,183.78,1,Desktop,Location 2,18-25
4,5,21,3,25.948584,4,10,59.51,1,Tablet,Location 4,18-25


In [49]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   userID           5000 non-null   int64   
 1   age              5000 non-null   int64   
 2   sessions         5000 non-null   int64   
 3   time_spent       5000 non-null   float64 
 4   pages_visited    5000 non-null   int64   
 5   cart_items       5000 non-null   int64   
 6   cart_value       5000 non-null   float64 
 7   checkout_status  5000 non-null   int64   
 8   device           5000 non-null   category
 9   location         5000 non-null   category
 10  age_segment      5000 non-null   category
dtypes: category(3), float64(2), int64(6)
memory usage: 327.8 KB


Now, we could perform ANOVA test to see if there is significant difference between the mean of checkout_status from different age_segments.

In [51]:
age_1 = dt[dt['age_segment'] == '18-25']['checkout_status']
age_2 = dt[dt['age_segment'] == '26-40']['checkout_status']
age_3 = dt[dt['age_segment'] == '41-55']['checkout_status']
age_4 = dt[dt['age_segment'] == '56+']['checkout_status']

F_stats, p_value = stats.f_oneway(age_1, age_2, age_3, age_4)

print('ANOVA Test')
print('F-statistics: {:.5f}'.format(F_stats))
print('P-value: {:.5f}'.format(p_value))

ANOVA Test
F-statistics: 1.44540
P-value: 0.22751


The high p-value of 0.22751 suggests that there is no significant difference between the mean of checkout_status from different groups.

#### Part three: 
#### Filter users by checked out and not checked out, then we would have two groups of users. Then, we perform a t-test on time_spent, and cart_values.

In [52]:
# Filter out two groups: dt_checkout (users who checked out), and dt_not_checkout (users who do not check out)
dt_checkout = dt[dt['checkout_status']==1]
dt_not_checkout = dt[dt['checkout_status']==0]

In [58]:
# Perform t-test on time spent
dt_checkout_tp = dt_checkout['time_spent']
dt_not_checkout_tp = dt_not_checkout['time_spent']

In [55]:
t_stats, p_val = stats.ttest_ind(dt_checkout_tp, dt_not_checkout_tp)

print('T test:')
print('T-statistics: {:.5f}'.format(t_stats))
print('P-value: {:.5f}'.format(p_val))

T test:
T-statistics: 29.36614
P-value: 0.00000


The P-value of 0.000 is very small, which is less than 0.05. This indicates that there is a significant difference on time spent on the website between the users who checkout and not checkout.

Further question is, which group spends more time on the website? Is it checkout or not checkout?

In [56]:
dt_checkout_tp.describe()

count    1362.000000
mean       31.317918
std         4.183834
min        25.000659
25%        27.602310
50%        30.943031
75%        34.567006
max        39.966537
Name: time_spent, dtype: float64

Average time spent for checkout users is 31 minutes.

In [57]:
dt_not_checkout_tp.describe()

count    3638.000000
mean       22.852522
std        10.325582
min         1.000000
25%        16.008221
50%        21.637722
75%        28.422609
max        90.000000
Name: time_spent, dtype: float64

Average time spent for not checkout users is 22 minutes.

Based on the mean value of time spent from checkout group and not checkout group. We could see that the average time spent from checkout group is higher than that of not checkout users.
This means that the more time the users spend on the website, the higher likelihood of checkout ratios.

Now, let's focus on the cart_values.

In [59]:
# Perform t-test on cart_values
dt_checkout_cv = dt_checkout['cart_value']
dt_not_checkout_cv = dt_not_checkout['cart_value']

In [60]:
t_stats, p_val = stats.ttest_ind(dt_checkout_cv, dt_not_checkout_cv)

print('T test:')
print('T-statistics: {:.5f}'.format(t_stats))
print('P-value: {:.5f}'.format(p_val))

T test:
T-statistics: -5.44258
P-value: 0.00000


The small P-value pf 0.000 indicates that there is a significant difference between the mean of cart_value from checkout group and not checkout group.

In [61]:
# Which group has a higher cart_value?
dt_checkout_cv.describe()

count    1362.000000
mean      140.824214
std        47.654924
min        43.220000
25%       103.357500
50%       136.290000
75%       174.375000
max       313.760000
Name: cart_value, dtype: float64

For checkout group:
Average cart value is 140 dollars. Highest is 313 dollars. Most of them falls in between 100 and 180.

In [62]:
dt_not_checkout_cv.describe()

count    3638.000000
mean      152.669915
std        74.841744
min        20.410000
25%        85.607500
50%       147.155000
75%       209.537500
max       427.400000
Name: cart_value, dtype: float64

For not checkout group
Average cart value is 152 dollars. Highest is 427 dollars. Most of the value falls in between 80-210.

Based on the insight above, the average cart value for not checkout group is more than that of checkout group. Maybe, the high value prevents the users from checking out the items.

#### Part four: Conclusion and Recommendation

#### Conclusion:

1. Age, device, and location does not yield significant impact on the checkout status.
2. Users spent more time on the website tends to make purchases and actually checked out in the end.
3. Users that had a relatively low cart values are likely to check out at the end of their browsing.


#### Recommendation:

1. Firstly, adding new and small features to the website that could appeal and attract more users.
For this step, the company could do a beta test. For example, do a (5%) pilot test, launch the new features in just a few time zones for a week.
2. After beta testing, gather website traffic information and improve the website or debugging any problems. Then, if everything works well. Do a 50% launch in most areas. 
3. Repetitively improve and gather user stats. Then, do a 75% launch and incrementally reach 100%.
4. For users who has 200+ or 300+ dollars of cart value in their carts, if the users do not check out for two or three weeks, the company could probably give them a 5-10% discounts on their purcahses.
This is to ensure customer loyalty and satisfaction.